## Reading of ASCII files created for cam diagnostics tool

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import os
from helper_funcs import read_file_custom, load_varinfo
import ipywidgets as ipw
from IPython import display
from copy import deepcopy
from traceback import format_exc
from collections import OrderedDict as od

VERBOSE = True

### 1. Paths and file definitions (PATHS)  

Please change accordingly if you execute this notebook on your local machine.

In [2]:
data_dir = "./data/michael_ascii_read/"
case_ok = "./data/from_ada/table_GLBL_ANN_obs_FIXED.asc"

michaels_excel = data_dir + "obs-comp-noresmversions.xlsx"
varinfo_csv = "./data/var_info.csv"

output_dir = "./output/"

### 2. Importing and editing supplementary information from Excel table (EXCEL)

Let's begin with reading the variable information from the excel table. Note that this is not strictly required but helps us below to display the results in a more intuitive manner, when analysing the data. The custom method that we use is:

In [3]:
help(load_varinfo)

Help on function load_varinfo in module helper_funcs:

load_varinfo(try_path, catch_excel_michael)
    Read short description strings for variables
    
    Load long names of variables. Tries to load information from csv file
    specified by input parameter ``try_path`` and if this fails, the information
    is imported from Michaels Excel table, in which case the csv file will be 
    created at location ``try_path``.
    
    Parameters
    ----------
    try_path : str
        location of csv file
    catch_excel_michael : str
        path to Michaels Excel
    
    Returns
    -------
    dict 
        dictionary containing all variable names (keys) and corresponding
        description strings (if applicable, else empty string)



Load the information.

In [4]:
var_info_dict = load_varinfo(varinfo_csv, michaels_excel)

In [5]:
from helper_funcs import save_varinfo_dict_csv, load_varinfo_dict_csv
from traceback import format_exc

class AddInfo(object):
    output = ipw.Output()
    def __init__(self, var_dict=None, csv_loc=None):
        self.csv_loc = csv_loc
        if not isinstance(var_dict, dict):
            self.load_csv()
        else:
            self.var_dict = var_dict
            
        self.init_widgets()
        self.init_actions()
        self.init_layout()
    
    def init_widgets(self):
        
        self.btn_update = ipw.Button(description='Update',
                                     tooltip=('Updates the current dictionary based on values in text fields'
                                              '(for further analysis, use Save csv button to write to CSV)'))
        self.btn_reload = ipw.Button(description='Reload',
                                     tooltip='Reloads information from file var_info.csv')
        self.btn_save = ipw.Button(description='Update and save',
                                     tooltip='Updates current selection and writes to CSV')
        
        self.btn_save.style.button_color = "lime"
        
        self.input_rows = []
        self.input_fields = {}
        
        for name,  val in self.var_dict.items():
            ipt = ipw.Text(value=val, placeholder='Insert new name',
                            disabled=False, min_width="200px")
            row = ipw.HBox([ipw.Label(name, minwidth="200px"), ipt])
            self.input_fields[name] = ipt
            self.input_rows.append(row)  
            
    def init_actions(self):
        self.btn_update.on_click(self.on_click_update)
        self.btn_reload.on_click(self.on_click_load_csv)
        self.btn_save.on_click(self.on_click_save)
        
    def init_layout(self):
        
        vbox_buttons = ipw.VBox([self.btn_reload,
                                 self.btn_update,
                                 self.btn_save])
        self.layout = ipw.HBox([ipw.VBox(self.input_rows), vbox_buttons, self.output])
        
    def on_click_update(self, b):
        self.apply_changes()
    
    def on_click_load_csv(self, b):
        self.load_csv()
        
    def on_click_save(self, b):
        self.save_csv()
    
    def save_csv(self):
        self.apply_changes()
        save_varinfo_dict_csv(self.var_dict, self.csv_loc)
        
    def load_csv(self):
        if self.csv_loc is None or not os.path.exists(self.csv_loc):
            raise IOError("Please provide path to csv file")
        try:
            self.var_dict = load_varinfo_dict_csv(self.csv_loc)
        except Exception as e:
            self.write_to_output(format_exc())
        for key, val in self.var_dict.items():
            self.input_fields[key].value = val
    
    def write_to_output(self, msg):
        self.output.append_display_data(msg)
        self.output
        
    def apply_changes(self):
        
        new = od()
        for key, edit in self.input_fields.items():
            new[key] = edit.value
        
        self.var_dict = new
        
    def __call__(self):
        return self.layout
    
adder = AddInfo(var_info_dict, varinfo_csv)
#show
adder()

In [6]:
var_info_dict = adder.var_dict
var_info_dict

OrderedDict([('RESTOM', 'TOmodel net flux'),
             ('RESSURF', 'SRF net flux'),
             ('RESTOA_CERES-EBAF', 'TOA  net flux'),
             ('RESTOA_ERBE', ''),
             ('SOLIN_CERES-EBAF', ''),
             ('SOLIN_CERES', ''),
             ('CLDTOT_ISCCP', 'Total cloud cover'),
             ('CLDTOT_CLOUDSAT', ''),
             ('FLDS_ISCCP', 'LW down SRF'),
             ('FLNS_ISCCP', 'LW net SRF'),
             ('FLUT_CERES-EBAF', 'LW up Top'),
             ('FLUT_CERES', ''),
             ('FLUT_ERBE', ''),
             ('FLUTC_CERES-EBAF', 'LW up Top Clearsky'),
             ('FLUTC_CERES', ''),
             ('FLUTC_ERBE', ''),
             ('FLNT_CAM', ''),
             ('FSDS_ISCCP', 'SW down SRF'),
             ('FSNS_ISCCP', 'SW net SRF'),
             ('FSNS_LARYEA', ''),
             ('FSNTOA_CERES', 'SW net TOA'),
             ('FSNTOA_ERBE', ''),
             ('FSNTOAC_CERES', 'SW net TOA clearsky'),
             ('FSNTOAC_ERBE', ''),
             ('FSNT

### 3. Search and load ASCII files, either using .asc or .webarchive file type (LOAD_FILE)

In [7]:
files = sorted(glob(data_dir + "*.webarchive"))

for file in files:
    print(file)
    
test_file = files[0]

print("TEST FILE: {}".format(os.path.basename(test_file)))

./data/michael_ascii_read/N1850C53CLM45L32_f09_tn11_191017 (yrs 71-100).webarchive
./data/michael_ascii_read/N1850_f09_tn14_230218 (yrs 1-20).webarchive
./data/michael_ascii_read/N1850_f19_tn14_r227_ctrl (yrs 185-215).webarchive
./data/michael_ascii_read/N1850_f19_tn14_r227_ctrl (yrs 310-340).webarchive
./data/michael_ascii_read/N1850_f19_tn14_r227_ctrl (yrs 80-110).webarchive
./data/michael_ascii_read/N1850_f19_tn14_r265_ctrl_20180411 (yrs 90-120).webarchive
TEST FILE: N1850C53CLM45L32_f09_tn11_191017 (yrs 71-100).webarchive


Try read first file as is with pandas

In [8]:
try:
    frame = pd.read_csv(test_file, encoding="latin-1")
except Exception as e:
    print(repr(e))
frame.head()

,bplist00Ñ_WebMainResourceÕ
0,_WebResourceTextEncodingName^WebResourceUR...
1,TEST CASE: N1850C53CLM45L32_f09_tn11_191017 (y...
2,CONTROL CASE: OBS data
3,Variable N1850C53CLM45L32_f09_tn11_191017 ...
4,...


This did not work (it basically did not separate the individual columns). The same is the case for the file that includes a whitespace at the problematic variables.

In [9]:
frame = pd.read_csv(case_ok)
frame.head()
frame.shape

(67, 1)

This did not really work since the data is not splitted by columns but includes one column containing the content of each row. The reading has to be done from scratch, especially also because there is some variables with too long names (e.g. L.25 and L. 28) that stick together the first two columns. 

This folder contains a file ``helper_funcs.py`` in which I defined a custom read function ``read_file_custom`` that can convert these files into pandas dataframes.

In [10]:
help(read_file_custom)

Help on function read_file_custom in module helper_funcs:

read_file_custom(fpath, var_info_dict=None, run_id=None, verbose=False)
    Custom ASCII conversion method 
    
    Parameters
    ----------
    fpath : str
        path to file location
    var_info_dict : dict
        optinal dictionary that contains description strings for each of the 
        variables (e.g. retrieved using :func:`read_var_info_michaels_excel`)
    run_id : str, int or dict
        string or integer that may be used as index specifying the model run 
        and that should be used in the Dataframe for the index specifying the 
        model run (only relevant if multiple files are loaded and concatenated 
        into one dataframe). If None, the "TEST CASE" ID, specified in the 
        file header is used for the index.
    verbose : bool
        if True, print output (defaults to False)
        
    Returns
    -------
    Dataframe 
        pandas data frame ready for further analysis. NOTE: the retu

Now load the first file using this function (without providing the optional parameter *var_info_dict*, i.e. the info from Michael's Excel sheet.

In [11]:
df = read_file_custom(test_file, verbose=VERBOSE)
df

Ignoring line: bplist00Ñ_WebMainResourceÕ	
Ignoring line: 
Ignoring line: 
Ignoring line: _WebResourceTextEncodingName^WebResourceURL_WebResourceFrameName_WebResourceData_WebResourceMIMETypeUUTF-8_http://ns2345k.web.sigma2.no/noresm_diagnostics/N1850C53CLM45L32_f09_tn11_191017/CAM_DIAG/yrs71to100-obs/set1/table_GLBL_ANN_obs.ascPO}<html><head></head><body><pre style="word-wrap: break-word; white-space: pre-wrap;">DIAG SET 1: ANN MEANS GLOBAL
Ignoring line:  
Ignoring line:  
Ignoring line:  
Problem case FSNTOA_CERES-EBAF
Problem case FSNTOAC_CERES-EBAF
Test case: N1850C53CLM45L32_f09_tn11_191017
Control case: OBS data


Flag  \
Run                              Years  Variable           Description          
N1850C53CLM45L32_f09_tn11_191017 71-100 RESTOM                          False   
                                        RESSURF                         False   
                                        RESTOA_CERES-EBAF               False   
                                        RESTOA_ERBE                     False   
                                        SOLIN_CERES-EBAF                False   
                                        SOLIN_CERES                     False   
                                        CLDTOT_ISCCP                    False   
                                        CLDTOT_CLOUDSAT                 False   
                                        FLDS_ISCCP                      False   
                                        FLNS_ISCCP                      False   
                                        FLUT_CERES-EBAF                 False   
                                        FLUT_CERES                      False   
                                        FLUT_ERBE                       False   
                                        FLUTC_CERES-EBAF                False   
                                        FLUTC_CERES                     False   
                                        FLUTC_ERBE                      False   
                                        FLNT_CAM                        False   
                                        FSDS_ISCCP                      False   
                                        FSNS_ISCCP                      False   
                                        FSNS_LARYEA                     False   
                                        FSNTOA_CERES-EBAF               False   
                                        FSNTOA_CERES                    False   
                                        FSNTOA_ERBE                     False   
                                        FSNTOAC_CERES-EBAF              False   
                                        FSNTOAC_CERES                   False   
                                        FSNTOAC_ERBE                    False   
                                        FSNT_CAM                        False   
                                        LHFLX_JRA25                     False   
                                        LHFLX_ERA40                     False   
                                        LHFLX_WHOI                      False   
...                                                                       ...   
                                        PRECT_GPCP                      False   
                                        PREH2O_NVAP                     False   
                                        PREH2O_AIRS                     False   
                                        PREH2O_JRA25                    False   
                                        PREH2O_ERAI                     False   
                                        PREH2O_ERA40                    False   
                                        PSL_JRA25                       False   
                                        PSL_ERAI                        False   
                                        SHFLX_JRA25                     False   
                                        SHFLX_NCEP                      False   
                                        SHFLX_LARYEA                    False   
                                        STRESS_MAG_ERS                  False   
                                        STRESS_MAG_LARYEA               False   
                                        STRESS_MAG_JRA25                False   
                                        SWCF_CERES-EBAF                 False   
                                        SWCF_CERES                      False   
                                        SWCF_ERBE                       False   
                       

That worked, you can see that the column *Description* is empty and that all flags are set `False`. Here in the default reading function the flag is set True, if the ``var_info_dict`` is provided and information for a given variable is available. We illustrate that in the following, our second example, where we load the corrected ascii file and provide the info dictionary that we imported from the Excel sheet. Furthermore, we use the optional input parameter `run_id` to shorten to shorten the width of the HTML table display of the dataframe.

In [12]:
df1 = read_file_custom(case_ok, var_info_dict, run_id="Run1", verbose=False)
df1

Flag     Model  \
Run  Years   Variable           Description                            
Run1 150-180 RESTOM             TOmodel net flux      True     0.020   
             RESSURF            SRF net flux          True     0.027   
             RESTOA_CERES-EBAF  TOA  net flux         True     2.109   
             RESTOA_ERBE                             False     2.109   
             SOLIN_CERES-EBAF                        False   340.200   
             SOLIN_CERES                             False   340.200   
             CLDTOT_ISCCP       Total cloud cover     True    70.746   
             CLDTOT_CLOUDSAT                         False    70.746   
             FLDS_ISCCP         LW down SRF           True   347.663   
             FLNS_ISCCP         LW net SRF            True    56.374   
             FLUT_CERES-EBAF    LW up Top             True   238.633   
             FLUT_CERES                              False   238.633   
             FLUT_ERBE                               False   238.633   
             FLUTC_CERES-EBAF   LW up Top Clearsky    True   264.795   
             FLUTC_CERES                             False   264.795   
             FLUTC_ERBE                              False   264.795   
             FLNT_CAM                                False   237.303   
             FSDS_ISCCP         SW down SRF           True   187.745   
             FSNS_ISCCP         SW net SRF            True   163.761   
             FSNS_LARYEA                             False   163.761   
             FSNTOA_CERES-EBAF                       False   240.742   
             FSNTOA_CERES       SW net TOA            True   240.742   
             FSNTOA_ERBE                             False   240.742   
             FSNTOAC_CERES-EBAF                      False   289.674   
             FSNTOAC_CERES      SW net TOA clearsky   True   289.674   
             FSNTOAC_ERBE                            False   289.674   
             FSNT_CAM                                False   237.323   
             LHFLX_JRA25        Lat Heat Flux         True    85.136   
             LHFLX_ERA40                             False    85.136   
             LHFLX_WHOI                              False    85.136   
...                                                    ...       ...   
             PRECT_GPCP         Precipitation         True     2.914   
             PREH2O_NVAP                             False    25.663   
             PREH2O_AIRS                             False    25.663   
             PREH2O_JRA25                            False    25.663   
             PREH2O_ERAI                             False    25.663   
             PREH2O_ERA40       Precipitable water    True    25.663   
             PSL_JRA25          SeaLev pressure       True  1011.611   
             PSL_ERAI                                False  1011.611   
             SHFLX_JRA25        Sens Heat Flux        True    22.225   
             SHFLX_NCEP                              False    22.225   
             SHFLX_LARYEA                            False    22.225   
             STRESS_MAG_ERS                          False     0.063   
             STRESS_MAG_LARYEA                       False     0.063   
             STRESS_MAG_JRA25                        False     0.063   
             SWCF_CERES-EBAF    SW Cloud Forc         True   -48.932   
             SWCF_CERES                              False   -48.932   
             SWCF_ERBE                               False   -48.932   
             AODVIS                                  False       NaN   
             AODDUST                                 False       NaN   
             SST_HADISST                             False    21.370   
             SST_HADISST_PI     SST preindustrial     True    21.690   
             SST_HADISST_PD                          False    21.359   
             TREFHT_LEGATES                          False   287.917   
             TREFHT_JR

As specified in the docstring of the reading method, the actual test case ID is stored (as attribute `test_case` in the Dataframe) in form of a dictionary that maps test_case with specified run_id:

In [13]:
print(df1.test_case)

N1850_f19_tn14_r265_ctrl_20180411


Reading worked, check, if both dataframes have the same dimension.

In [14]:
print(df.shape)
print(df1.shape)

(63, 5)
(63, 5)


You can change the appearance of the way a DataFrame is displayed. For instance ...

In [15]:
df1.style.background_gradient(cmap="GnBu", low=0.5, high=0.5, axis=0).highlight_null("red")

/home/jonasg/anaconda3/lib/python3.6/site-packages/matplotlib/colors.py:489: RuntimeWarning: invalid value encountered in less
  np.copyto(xa, -1, where=xa < 0.0)


... applies a column based colour gradient based on the cell values in each column (similar to Excel feature *Conditional formatting*). Here, we provided a colormap of our choice (see [here](https://matplotlib.org/examples/color/colormaps_reference.html) for options). The input parameters `low` and `high` are optional, and can be interpreted as an percentage specification of the colour range used from the colourbar for the display (i.e. here, we use 0.5, i.e. 50% both for the lower and upper end).

[Follow this link](https://pandas.pydata.org/pandas-docs/stable/style.html) for more details related to pandas styling. 

### 4. Importing multiple result files and concatenating them into one Dataframe (LOAD_FILES)

Now we have a method that can import the results from a single run into a datframe that can be used for further analysis. In the following, we basically do this for all available files and put the results into one big `Dataframe`. 

To do this, a custom method `read_and_merge_all` was defined in [helper_funcs.py](https://github.com/jgliss/my_py3_scripts/blob/master/notebooks/helper_funcs.py). 

The following cells show, how this method may be used to either keep the original test case IDs as index or to replace them with a shorter version. 

In [16]:
from helper_funcs import read_and_merge_all
merged = read_and_merge_all(file_list=files, var_info_dict=var_info_dict)
merged

/home/jonasg/github/my_notebooks/helper_funcs.py:221: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.test_case = pd.Series()


Flag  \
Run                               Years  Variable           Description                  
N1850C53CLM45L32_f09_tn11_191017  71-100 AODDUST                                 False   
                                         AODVIS                                  False   
                                         CLDTOT_CLOUDSAT                         False   
                                         CLDTOT_ISCCP       Total cloud cover     True   
                                         FLDS_ISCCP         LW down SRF           True   
                                         FLNS_ISCCP         LW net SRF            True   
                                         FLNT_CAM                                False   
                                         FLUTC_CERES                             False   
                                         FLUTC_CERES-EBAF   LW up Top Clearsky    True   
                                         FLUTC_ERBE                              False   
                                         FLUT_CERES                              False   
                                         FLUT_CERES-EBAF    LW up Top             True   
                                         FLUT_ERBE                               False   
                                         FSDS_ISCCP         SW down SRF           True   
                                         FSNS_ISCCP         SW net SRF            True   
                                         FSNS_LARYEA                             False   
                                         FSNTOAC_CERES      SW net TOA clearsky   True   
                                         FSNTOAC_CERES-EBAF                      False   
                                         FSNTOAC_ERBE                            False   
                                         FSNTOA_CERES       SW net TOA            True   
                                         FSNTOA_CERES-EBAF                       False   
                                         FSNTOA_ERBE                             False   
                                         FSNT_CAM                                False   
                                         LHFLX_ERA40                             False   
                                         LHFLX_JRA25        Lat Heat Flux         True   
                                         LHFLX_WHOI                              False   
                                         LWCF_CERES                              False   
                                         LWCF_CERES-EBAF    LW Cloud Forc         True   
                                         LWCF_ERBE                               False   
                                         PRECT_GPCP         Precipitation         True   
...                                                                                ...   
N1850_f19_tn14_r265_ctrl_20180411 90-120 PREH2O_JRA25                            False   
                                         PREH2O_NVAP                             False   
                                         PSL_ERAI                                False   
                                         PSL_JRA25          SeaLev pressure       True   
                                         RESSURF            SRF net flux          True   
                                         RESTOA_CERES-EBAF  TOA  net flux         True   
                                         RESTOA_ERBE                             False   
                                         RESTOM             TOmodel net flux      True   
                                         SHFLX_JRA25        Sens Heat Flux        True   
                                         SHFLX_LARYEA                            False   
                                         SHFLX_NCEP                              False   
                                         SOLIN_CERES                             False   
                                

This is rather unhandy, since the names of the run IDs are rather long. This can be changed directly when loading the Dataframe:

In [17]:
merged = read_and_merge_all(file_list=files, var_info_dict=var_info_dict, replace_runid_prefix="Run")
merged

/home/jonasg/github/my_notebooks/helper_funcs.py:159: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.test_case = pd.Series(mapping)


Flag     Model  \
Run  Years   Variable           Description                            
Run1 185-215 AODDUST                                 False       NaN   
             AODVIS                                  False       NaN   
             CLDTOT_CLOUDSAT                         False    68.543   
             CLDTOT_ISCCP       Total cloud cover     True    68.543   
             FLDS_ISCCP         LW down SRF           True   353.861   
             FLNS_ISCCP         LW net SRF            True    56.249   
             FLNT_CAM                                False   240.640   
             FLUTC_CERES                             False   267.090   
             FLUTC_CERES-EBAF   LW up Top Clearsky    True   267.090   
             FLUTC_ERBE                              False   267.090   
             FLUT_CERES                              False   241.972   
             FLUT_CERES-EBAF    LW up Top             True   241.972   
             FLUT_ERBE                               False   241.972   
             FSDS_ISCCP         SW down SRF           True   190.458   
             FSNS_ISCCP         SW net SRF            True   166.962   
             FSNS_LARYEA                             False   166.962   
             FSNTOAC_CERES      SW net TOA clearsky   True   290.330   
             FSNTOAC_CERES-EBAF                      False   290.330   
             FSNTOAC_ERBE                            False   290.330   
             FSNTOA_CERES       SW net TOA            True   244.525   
             FSNTOA_CERES-EBAF                       False   244.525   
             FSNTOA_ERBE                             False   244.525   
             FSNT_CAM                                False   241.108   
             LHFLX_ERA40                             False    87.926   
             LHFLX_JRA25        Lat Heat Flux         True    87.926   
             LHFLX_WHOI                              False    87.926   
             LWCF_CERES                              False    25.119   
             LWCF_CERES-EBAF    LW Cloud Forc         True    25.119   
             LWCF_ERBE                               False    25.119   
             PRECT_GPCP         Precipitation         True     3.010   
...                                                    ...       ...   
Run4 71-100  PREH2O_JRA25                            False    23.974   
             PREH2O_NVAP                             False    23.974   
             PSL_ERAI                                False  1011.281   
             PSL_JRA25          SeaLev pressure       True  1011.281   
             RESSURF            SRF net flux          True    -0.489   
             RESTOA_CERES-EBAF  TOA  net flux         True     1.529   
             RESTOA_ERBE                             False     1.529   
             RESTOM             TOmodel net flux      True    -0.489   
             SHFLX_JRA25        Sens Heat Flux        True    20.445   
             SHFLX_LARYEA                            False    20.445   
             SHFLX_NCEP                              False    20.445   
             SOLIN_CERES                             False   340.206   
             SOLIN_CERES-EBAF                        False   340.206   
             SST_HADISST                             False    19.826   
             SST_HADISST_PD                          False    19.830   
             SST_HADISST_PI     SST preindustrial     True    20.180   
             STRESS_MAG_ERS                          False     0.068   
             STRESS_MAG_JRA25                        False     0.068   
             STRESS_MAG_LARYEA                       False     0.068   
             SWCF_CERES                              False   -48.323   
             SWCF_CERES-EBAF    SW Cloud Forc         True   -48.323   
             SWCF_ERBE                               False   -48.323   
             TREFHT_JRA25       2m Temp               True   286.507   
             TREFHT_LE

Call:

In [18]:
merged.test_case

Run1             N1850_f19_tn14_r227_ctrl
Run2    N1850_f19_tn14_r265_ctrl_20180411
Run3                N1850_f09_tn14_230218
Run4     N1850C53CLM45L32_f09_tn11_191017
dtype: object

to get the corresponding test_case IDs.

### 5. Rearranging and restructuring of the imported data (REARRANGE)

In the following cell, you can interacively select which Variables you wish to keep for further analysis. Preselected are the variables that are flagged.

#### 5.1 Interactive selection of variables (IA_VAR)

In [19]:
# TODO: 
# 1. Move into helper_funcs.py when finalised
# 2. Inlcude reordering
# 3. Write documentation
class SelectVariable():
    output = ipw.Output()
    def __init__(self, df):
        df.sort_index(inplace=True)
        self.df = df
        self.vals = tuple(self.df.index.levels[2].values)
        
        self._df_edit = self.df
        
        self.init_layout()
        self.init_widgets()
        self.init_actions()
        self.init_display()
        
        self.print_current(1)
        self.crop_selection()
        self.disp_current()
    
    @property
    def df_edit(self):
        return deepcopy(self._df_edit)
    
    @property
    def flagged_vars(self):
        return list(self.df[self.df.Flag].index.get_level_values("Variable").unique().values)
    
    def init_widgets(self):
        
        self.btn_unselect_all = ipw.Button(description='Unselect all')
        self.btn_select_all = ipw.Button(description='Select all')
        self.btn_flagged = ipw.Button(description="Flagged")
        self.btn_apply = ipw.Button(description='Apply')
        self.btn_apply.style.button_color = 'lime'

        self.var_selector = ipw.SelectMultiple(description="Variables", 
                                               options=self.vals, 
                                               value=self.flagged_vars, 
                                               min_width='150px',
                                               layout=self.box_layout)
        
        self.current_disp = ipw.Textarea(value='', 
                                         description='Current:', 
                                         disabled=True, 
                                         layout=self.box_layout)
        #self.output = ipw.Output()
        
    def init_actions(self):
        #what happens when the state of the selection is changed (display current selection)
        self.var_selector.observe(self.print_current)
        #what happens when buttons are clicked
        self.btn_select_all.on_click(self.on_select_all_vars_clicked)
        self.btn_unselect_all.on_click(self.on_unselect_all_vars_clicked)
        self.btn_flagged.on_click(self.on_flagged_clicked)
        self.btn_apply.on_click(self.on_click_apply)
        
    def init_layout(self):
        self.box_layout = ipw.Layout(flex='0 1 auto', height='250px', min_height='150px', width='auto')
        self.disp_layout = ipw.Layout(flex='0 1 auto', height='250px', min_height='150px', width='auto')
    
    def init_display(self):
        self.btns = ipw.VBox([self.btn_select_all, 
                              self.btn_unselect_all,
                              self.btn_flagged,
                              ipw.Label(),
                              self.btn_apply])
    
        self.edit_area = ipw.HBox([self.var_selector, 
                                   self.current_disp, 
                                   self.btns])
        
        self.layout = ipw.VBox([self.edit_area, self.output])
    
    def on_unselect_all_vars_clicked(self, b):
        self.unselect_all()
    
    def on_select_all_vars_clicked(self, b):
        self.select_all()
    
    def on_flagged_clicked(self, b):
        self.select_flagged()
        
    def unselect_all(self):
        self.var_selector.value = ()
    
    def select_all(self):
        self.var_selector.value = self.var_selector.options
    
    def select_flagged(self):
        self.var_selector.value = self.flagged_vars
        
    def disp_current(self):
        self.output.clear_output()
        #self.output.append_display_data(ipw.Label("PREVIEW current selection", fontsize=22))
        self.output.append_display_data(self._df_edit.head().style.set_caption("PREVIEW HEAD"))
        self.output
        
    def crop_selection(self):
        idx = pd.IndexSlice
        try:
            self._df_edit = self.df.loc[idx[:, :, self.var_selector.value, :], :]
        except Exception as e:
            print("WARNING: failed to extract selection.\nTraceback {}".format(format_exc()))
    
    def on_click_apply(self, b):
        self.crop_selection()
        self.disp_current()
        
    def print_current(self, b):
        s=""
        for item in self.var_selector.value:
            s += "{}\n".format(item)
        self.current_disp.value = s
    
    def __repr__(self):
        return repr(self.layout)
    
    def __call__(self):
        return self.layout
    
selector = SelectVariable(df=merged)
#show
selector()

In [20]:
selector.df_edit

Flag     Model       Obs  \
Run  Years   Variable          Description                                     
Run1 185-215 CLDTOT_ISCCP      Total cloud cover    True    68.543    66.800   
             FLDS_ISCCP        LW down SRF          True   353.861   343.347   
             FLNS_ISCCP        LW net SRF           True    56.249    49.425   
             FLUTC_CERES-EBAF  LW up Top Clearsky   True   267.090   266.051   
             FLUT_CERES-EBAF   LW up Top            True   241.972   239.574   
             FSDS_ISCCP        SW down SRF          True   190.458   189.390   
             FSNS_ISCCP        SW net SRF           True   166.962   165.893   
             FSNTOAC_CERES     SW net TOA clearsky  True   290.330   294.702   
             FSNTOA_CERES      SW net TOA           True   244.525   244.691   
             LHFLX_JRA25       Lat Heat Flux        True    87.926    87.935   
             LWCF_CERES-EBAF   LW Cloud Forc        True    25.119    26.476   
             PRECT_GPCP        Precipitation        True     3.010     2.674   
             PREH2O_ERA40      Precipitable water   True    28.091    24.948   
             PSL_JRA25         SeaLev pressure      True  1011.771  1011.280   
             RESSURF           SRF net flux         True     0.474     0.000   
             RESTOA_CERES-EBAF TOA  net flux        True     2.553     0.992   
             RESTOM            TOmodel net flux     True     0.468     0.000   
             SHFLX_JRA25       Sens Heat Flux       True    22.314    19.372   
             SST_HADISST_PI    SST preindustrial    True    22.582    20.365   
             SWCF_CERES-EBAF   SW Cloud Forc        True   -45.805   -47.071   
             TREFHT_JRA25      2m Temp              True   289.039   287.605   
             TS_LAND_NCEP      2m Temp land         True   284.920   281.155   
             TS_NCEP           2m Temp              True   290.262   287.683   
             U_200_JRA25       Zonal Wind JRA25     True    18.774    15.565   
             Z3_500_JRA25      Geop Height  JRA25   True    56.879    56.601   
     310-340 CLDTOT_ISCCP      Total cloud cover    True    68.234    66.800   
             FLDS_ISCCP        LW down SRF          True   354.846   343.347   
             FLNS_ISCCP        LW net SRF           True    56.272    49.425   
             FLUTC_CERES-EBAF  LW up Top Clearsky   True   267.477   266.051   
             FLUT_CERES-EBAF   LW up Top            True   242.502   239.574   
...                                                  ...       ...       ...   
Run3 1-20    TREFHT_JRA25      2m Temp              True   287.342   287.605   
             TS_LAND_NCEP      2m Temp land         True   282.392   281.155   
             TS_NCEP           2m Temp              True   288.559   287.683   
             U_200_JRA25       Zonal Wind JRA25     True    17.390    15.565   
             Z3_500_JRA25      Geop Height  JRA25   True    56.511    56.601   
Run4 71-100  CLDTOT_ISCCP      Total cloud cover    True    63.621    66.800   
             FLDS_ISCCP        LW down SRF          True   338.280   343.347   
             FLNS_ISCCP        LW net SRF           True    55.819    49.425   
             FLUTC_CERES-EBAF  LW up Top Clearsky   True   261.783   266.051   
             FLUT_CERES-EBAF   LW up Top            True   238.148   239.574   
             FSDS_ISCCP        SW down SRF          True   187.801   189.390   
             FSNS_ISCCP        SW net SRF           True   163.679   165.893   
             FSNTOAC_CERES     SW net TOA clearsky  True   287.999   294.702   
             FSNTOA_CERES      SW net TOA           True   239.677   244.691   
             LHFLX_JRA25       Lat Heat Flux        True    87.904    87.935   
             LWCF_CERES-EBAF   LW Cloud Forc        True    23.635    26.476   
             PRECT_GPCP        Precipitation        True     3.003     2.674   
             PREH2O_ERA40      Precipitable water   

Now access the current selection and continue.

#### 5.2 Interactive index renaming tool (IA_RENAME)

In the following, an interactive widget is defined, that allows for renaming of the runs.

In [21]:
class IndexRenamer(object):
    output = ipw.Output()
    def __init__(self, df, level=0, suggestions=[]):
        self.df = df
        self._df_edit = df
        self.level = level
        
        self.suggestions = suggestions
      
        self.init_widgets()
        self.init_actions()
        self.init_layout()
        
        self.renamed_info = od()
        
    @property
    def names(self):
        return sorted(self.df.index.get_level_values(self.level).unique().values)
        
    @property
    def df_edit(self):
        return deepcopy(self._df_edit)
    
    def init_widgets(self):
        
        self.btn_apply = ipw.Button(description='Apply')
        self.btn_apply.style.button_color = "lime"
        
        self.input_rows = []
        self.input_fields = []
        
        for i, name in enumerate(self.names):
            try:
                val = self.suggestions[i]
            except:
                val = name
            ipt = ipw.Text(value=val, placeholder='Insert new name',
                            disabled=False)
            row = ipw.HBox([ipw.Label(name, width="200px"), ipt])
            self.input_fields.append(ipt)
            self.input_rows.append(row)
                                      
    def init_actions(self):
        #what happens when the state of the selection is changed (display current selection)
        self.btn_apply.on_click(self.on_click_apply)
        
    def init_layout(self):
        
        edit_area = ipw.HBox([ipw.VBox(self.input_rows), self.btn_apply])
        self.layout = ipw.VBox([edit_area, self.output])
        
    def on_click_apply(self, b):
        self.apply_changes()
        
    def disp_current(self):
        self.output.clear_output()
        #self.output.append_display_data(ipw.Label("PREVIEW current selection", fontsize=22))
        self.output.append_display_data(self._df_edit.style.set_caption("PREVIEW"))
        self.output
        
    def apply_changes(self):
        
        df = self.df 
        mapping = od()
        
        for i, name in enumerate(self.names):
            repl = str(self.input_fields[i].value)
            mapping[name] = repl
        self._df_edit = df.rename(index=mapping, level=self.level)
        
        self.disp_current()
        
    def __call__(self):
        return self.layout

selection  = selector.df_edit
selection

Flag     Model       Obs  \
Run  Years   Variable          Description                                     
Run1 185-215 CLDTOT_ISCCP      Total cloud cover    True    68.543    66.800   
             FLDS_ISCCP        LW down SRF          True   353.861   343.347   
             FLNS_ISCCP        LW net SRF           True    56.249    49.425   
             FLUTC_CERES-EBAF  LW up Top Clearsky   True   267.090   266.051   
             FLUT_CERES-EBAF   LW up Top            True   241.972   239.574   
             FSDS_ISCCP        SW down SRF          True   190.458   189.390   
             FSNS_ISCCP        SW net SRF           True   166.962   165.893   
             FSNTOAC_CERES     SW net TOA clearsky  True   290.330   294.702   
             FSNTOA_CERES      SW net TOA           True   244.525   244.691   
             LHFLX_JRA25       Lat Heat Flux        True    87.926    87.935   
             LWCF_CERES-EBAF   LW Cloud Forc        True    25.119    26.476   
             PRECT_GPCP        Precipitation        True     3.010     2.674   
             PREH2O_ERA40      Precipitable water   True    28.091    24.948   
             PSL_JRA25         SeaLev pressure      True  1011.771  1011.280   
             RESSURF           SRF net flux         True     0.474     0.000   
             RESTOA_CERES-EBAF TOA  net flux        True     2.553     0.992   
             RESTOM            TOmodel net flux     True     0.468     0.000   
             SHFLX_JRA25       Sens Heat Flux       True    22.314    19.372   
             SST_HADISST_PI    SST preindustrial    True    22.582    20.365   
             SWCF_CERES-EBAF   SW Cloud Forc        True   -45.805   -47.071   
             TREFHT_JRA25      2m Temp              True   289.039   287.605   
             TS_LAND_NCEP      2m Temp land         True   284.920   281.155   
             TS_NCEP           2m Temp              True   290.262   287.683   
             U_200_JRA25       Zonal Wind JRA25     True    18.774    15.565   
             Z3_500_JRA25      Geop Height  JRA25   True    56.879    56.601   
     310-340 CLDTOT_ISCCP      Total cloud cover    True    68.234    66.800   
             FLDS_ISCCP        LW down SRF          True   354.846   343.347   
             FLNS_ISCCP        LW net SRF           True    56.272    49.425   
             FLUTC_CERES-EBAF  LW up Top Clearsky   True   267.477   266.051   
             FLUT_CERES-EBAF   LW up Top            True   242.502   239.574   
...                                                  ...       ...       ...   
Run3 1-20    TREFHT_JRA25      2m Temp              True   287.342   287.605   
             TS_LAND_NCEP      2m Temp land         True   282.392   281.155   
             TS_NCEP           2m Temp              True   288.559   287.683   
             U_200_JRA25       Zonal Wind JRA25     True    17.390    15.565   
             Z3_500_JRA25      Geop Height  JRA25   True    56.511    56.601   
Run4 71-100  CLDTOT_ISCCP      Total cloud cover    True    63.621    66.800   
             FLDS_ISCCP        LW down SRF          True   338.280   343.347   
             FLNS_ISCCP        LW net SRF           True    55.819    49.425   
             FLUTC_CERES-EBAF  LW up Top Clearsky   True   261.783   266.051   
             FLUT_CERES-EBAF   LW up Top            True   238.148   239.574   
             FSDS_ISCCP        SW down SRF          True   187.801   189.390   
             FSNS_ISCCP        SW net SRF           True   163.679   165.893   
             FSNTOAC_CERES     SW net TOA clearsky  True   287.999   294.702   
             FSNTOA_CERES      SW net TOA           True   239.677   244.691   
             LHFLX_JRA25       Lat Heat Flux        True    87.904    87.935   
             LWCF_CERES-EBAF   LW Cloud Forc        True    23.635    26.476   
             PRECT_GPCP        Precipitation        True     3.003     2.674   
             PREH2O_ERA40      Precipitable water   

In [39]:
class IndexRenamer(object):
    output = ipw.Output()
    def __init__(self, df, level=0, suggestions=[]):
        self.df = df
        self._df_edit = df
        self.level = level
        
        self.suggestions = suggestions
      
        self.init_widgets()
        self.init_actions()
        self.init_layout()
        
        self.renamed_info = od()
        
    @property
    def names(self):
        #return sorted(self.df.index.get_level_values(self.level).unique().values)
        return self.df.index.get_level_values(self.level).unique().values
    @property
    def df_edit(self):
        return deepcopy(self._df_edit)
    
    def init_widgets(self):
        
        self.btn_apply = ipw.Button(description='Apply')
        self.btn_apply.style.button_color = "lime"
        
        self.input_rows = []
        self.input_fields = []
        
        for i, name in enumerate(self.names):
            try:
                val = self.suggestions[i]
            except:
                val = name
            ipt = ipw.Text(value=val, placeholder='Insert new name',
                            disabled=False)
            row = ipw.HBox([ipw.Label(name, width="200px"), ipt])
            self.input_fields.append(ipt)
            self.input_rows.append(row)
                                      
    def init_actions(self):
        #what happens when the state of the selection is changed (display current selection)
        self.btn_apply.on_click(self.on_click_apply)
        
    def init_layout(self):
        
        edit_area = ipw.HBox([ipw.VBox(self.input_rows), self.btn_apply])
        self.layout = ipw.VBox([edit_area, self.output])
        
    def on_click_apply(self, b):
        self.apply_changes()
        
    def disp_current(self):
        self.output.clear_output()
        #self.output.append_display_data(ipw.Label("PREVIEW current selection", fontsize=22))
        self.output.append_display_data(self._df_edit.style.set_caption("PREVIEW"))
        self.output
        
    def apply_changes(self):
        
        df = self.df 
        mapping = od()
        
        for i, name in enumerate(self.names):
            repl = str(self.input_fields[i].value)
            mapping[name] = repl
        self._df_edit = df.rename(index=mapping, level=self.level)
        
        self.disp_current()
        
    def __call__(self):
        return self.layout

renamer = IndexRenamer(selection)
renamer()

Now, update the current dataframe for further usage.

In [23]:
selection = renamer.df_edit
selection

Flag     Model       Obs  \
Run  Years   Variable          Description                                     
Run1 185-215 CLDTOT_ISCCP      Total cloud cover    True    68.543    66.800   
             FLDS_ISCCP        LW down SRF          True   353.861   343.347   
             FLNS_ISCCP        LW net SRF           True    56.249    49.425   
             FLUTC_CERES-EBAF  LW up Top Clearsky   True   267.090   266.051   
             FLUT_CERES-EBAF   LW up Top            True   241.972   239.574   
             FSDS_ISCCP        SW down SRF          True   190.458   189.390   
             FSNS_ISCCP        SW net SRF           True   166.962   165.893   
             FSNTOAC_CERES     SW net TOA clearsky  True   290.330   294.702   
             FSNTOA_CERES      SW net TOA           True   244.525   244.691   
             LHFLX_JRA25       Lat Heat Flux        True    87.926    87.935   
             LWCF_CERES-EBAF   LW Cloud Forc        True    25.119    26.476   
             PRECT_GPCP        Precipitation        True     3.010     2.674   
             PREH2O_ERA40      Precipitable water   True    28.091    24.948   
             PSL_JRA25         SeaLev pressure      True  1011.771  1011.280   
             RESSURF           SRF net flux         True     0.474     0.000   
             RESTOA_CERES-EBAF TOA  net flux        True     2.553     0.992   
             RESTOM            TOmodel net flux     True     0.468     0.000   
             SHFLX_JRA25       Sens Heat Flux       True    22.314    19.372   
             SST_HADISST_PI    SST preindustrial    True    22.582    20.365   
             SWCF_CERES-EBAF   SW Cloud Forc        True   -45.805   -47.071   
             TREFHT_JRA25      2m Temp              True   289.039   287.605   
             TS_LAND_NCEP      2m Temp land         True   284.920   281.155   
             TS_NCEP           2m Temp              True   290.262   287.683   
             U_200_JRA25       Zonal Wind JRA25     True    18.774    15.565   
             Z3_500_JRA25      Geop Height  JRA25   True    56.879    56.601   
     310-340 CLDTOT_ISCCP      Total cloud cover    True    68.234    66.800   
             FLDS_ISCCP        LW down SRF          True   354.846   343.347   
             FLNS_ISCCP        LW net SRF           True    56.272    49.425   
             FLUTC_CERES-EBAF  LW up Top Clearsky   True   267.477   266.051   
             FLUT_CERES-EBAF   LW up Top            True   242.502   239.574   
...                                                  ...       ...       ...   
Run3 1-20    TREFHT_JRA25      2m Temp              True   287.342   287.605   
             TS_LAND_NCEP      2m Temp land         True   282.392   281.155   
             TS_NCEP           2m Temp              True   288.559   287.683   
             U_200_JRA25       Zonal Wind JRA25     True    17.390    15.565   
             Z3_500_JRA25      Geop Height  JRA25   True    56.511    56.601   
Run4 71-100  CLDTOT_ISCCP      Total cloud cover    True    63.621    66.800   
             FLDS_ISCCP        LW down SRF          True   338.280   343.347   
             FLNS_ISCCP        LW net SRF           True    55.819    49.425   
             FLUTC_CERES-EBAF  LW up Top Clearsky   True   261.783   266.051   
             FLUT_CERES-EBAF   LW up Top            True   238.148   239.574   
             FSDS_ISCCP        SW down SRF          True   187.801   189.390   
             FSNS_ISCCP        SW net SRF           True   163.679   165.893   
             FSNTOAC_CERES     SW net TOA clearsky  True   287.999   294.702   
             FSNTOA_CERES      SW net TOA           True   239.677   244.691   
             LHFLX_JRA25       Lat Heat Flux        True    87.904    87.935   
             LWCF_CERES-EBAF   LW Cloud Forc        True    23.635    26.476   
             PRECT_GPCP        Precipitation        True     3.003     2.674   
             PREH2O_ERA40      Precipitable water   

#### 5.3 Reshaping of table (make it wider for readibility) (RESHAPE)
 
For visualisation this display requires a lot of scrolling. We can make the table `wider` by unstacking certain indices, e.g. the two outermost indices `Run` and `Years`.

In [24]:
class ReshapeAndSelect(object):
    """Widget that can be used to reshape a Dataframe and select individual data columns"""
    output = ipw.Output()
    def __init__(self, df):
        
        self.df = df
        self._df_edit = df
        
        self.index_names = df.index.names
        self.col_names = df.columns
    
    @property
    def df_edit(self):
        return deepcopy(self._df_edit)
    
    @property
    def flagged_vars(self):
        return list(self.df[self.df.Flag].index.get_level_values("Variable").unique().values)
    
    def init_widgets(self):
    
        self.btn_unselect_all = ipw.Button(description='Unselect all')
        self.btn_select_all = ipw.Button(description='Select all')
        self.btn_flagged = ipw.Button(description="Flagged")
        self.btn_apply = ipw.Button(description='Apply')
        self.btn_apply.style.button_color = 'lime'

        self.var_selector = ipw.SelectMultiple(description="Variables", 
                                               options=self.vals, 
                                               value=self.flagged_vars, 
                                               min_width='150px',
                                               layout=self.box_layout)
        
        self.current_disp = ipw.Textarea(value='', 
                                         description='Current:', 
                                         disabled=True, 
                                         layout=self.box_layout)
        #self.output = ipw.Output()
        
    def init_actions(self):
        #what happens when the state of the selection is changed (display current selection)
        self.var_selector.observe(self.print_current)
        #what happens when buttons are clicked
        self.btn_select_all.on_click(self.on_select_all_vars_clicked)
        self.btn_unselect_all.on_click(self.on_unselect_all_vars_clicked)
        self.btn_flagged.on_click(self.on_flagged_clicked)
        self.btn_apply.on_click(self.on_click_apply)
    
    def init_display(self):
        self.btns = ipw.VBox([self.btn_select_all, 
                              self.btn_unselect_all,
                              self.btn_flagged,
                              ipw.Label(),
                              self.btn_apply])
    
        self.edit_area = ipw.HBox([self.var_selector, 
                                   self.current_disp, 
                                   self.btns])
        
        self.layout = ipw.VBox([self.edit_area, self.output])
    
    def on_unselect_all_vars_clicked(self, b):
        self.unselect_all()
    
    def on_select_all_vars_clicked(self, b):
        self.select_all()
    
    def on_flagged_clicked(self, b):
        self.select_flagged()
        
    def unselect_all(self):
        self.var_selector.value = ()
    
    def select_all(self):
        self.var_selector.value = self.var_selector.options
    
    def select_flagged(self):
        self.var_selector.value = self.flagged_vars
        
    def disp_current(self):
        self.output.clear_output()
        #self.output.append_display_data(ipw.Label("PREVIEW current selection", fontsize=22))
        self.output.append_display_data(self._df_edit.head().style.set_caption("PREVIEW HEAD"))
        self.output
        
    def crop_selection(self):
        idx = pd.IndexSlice
        try:
            self._df_edit = self.df.loc[idx[:, :, self.var_selector.value, :], :]
        except Exception as e:
            print("WARNING: failed to extract selection.\nTraceback {}".format(format_exc()))
    
    def on_click_apply(self, b):
        self.crop_selection()
        self.disp_current()
        
    def print_current(self, b):
        s=""
        for item in self.var_selector.value:
            s += "{}\n".format(item)
        self.current_disp.value = s
    
    def __repr__(self):
        return repr(self.layout)
    
    def __call__(self):
        return self.layout
    

In [57]:
selection_unstacked = selection.unstack(["Run", "Years"])
selection_unstacked.stack(["Run", "Years"])

Bias  Flag    Model  \
Variable         Description        Run  Years                            
CLDTOT_ISCCP     Total cloud cover  Run1 185-215   1.744  True   68.543   
                                         310-340   1.435  True   68.234   
                                         80-110    2.157  True   68.956   
                                    Run2 90-120    3.947  True   70.747   
                                    Run3 1-20      1.786  True   68.586   
                                    Run4 71-100   -3.179  True   63.621   
FLDS_ISCCP       LW down SRF        Run1 185-215  10.514  True  353.861   
                                         310-340  11.499  True  354.846   
                                         80-110    5.162  True  348.508   
                                    Run2 90-120    4.507  True  347.854   
                                    Run3 1-20     -1.799  True  341.547   
                                    Run4 71-100   -5.066  True  338.280   
FLNS_ISCCP       LW net SRF         Run1 185-215   6.824  True   56.249   
                                         310-340   6.847  True   56.272   
                                         80-110    7.167  True   56.592   
                                    Run2 90-120    6.925  True   56.350   
                                    Run3 1-20      9.699  True   59.124   
                                    Run4 71-100    6.394  True   55.819   
FLUTC_CERES-EBAF LW up Top Clearsky Run1 185-215   1.039  True  267.090   
                                         310-340   1.426  True  267.477   
                                         80-110   -0.786  True  265.265   
                                    Run2 90-120   -1.182  True  264.869   
                                    Run3 1-20     -0.986  True  265.065   
                                    Run4 71-100   -4.268  True  261.783   
FLUT_CERES-EBAF  LW up Top          Run1 185-215   2.398  True  241.972   
                                         310-340   2.928  True  242.502   
                                         80-110    0.660  True  240.235   
                                    Run2 90-120   -0.832  True  238.742   
                                    Run3 1-20      1.704  True  241.278   
                                    Run4 71-100   -1.426  True  238.148   
...                                                  ...   ...      ...   
TREFHT_JRA25     2m Temp            Run1 185-215   1.434  True  289.039   
                                         310-340   1.615  True  289.220   
                                         80-110    0.431  True  288.036   
                                    Run2 90-120    0.344  True  287.949   
                                    Run3 1-20     -0.263  True  287.342   
                                    Run4 71-100   -1.098  True  286.507   
TS_LAND_NCEP     2m Temp land       Run1 185-215   3.765  True  284.920   
                                         310-340   3.945  True  285.100   
                                         80-110    2.181  True  283.336   
                                    Run2 90-120    2.153  True  283.308   
                                    Run3 1-20      1.237  True  282.392   
                                    Run4 71-100   -0.524  True  280.631   
TS_NCEP          2m Temp            Run1 185-215   2.579  True  290.262   
                                         310-340   2.767  True  290.450   
                                         80-110    1.564  True  289.247   
                                    Run2 90-120    1.518  True  289.201   
                                    Run3 1-20      0.876  True  288.559   
                                    Run4 71-100   -0.346  True  287.337   
U_200_JRA25      Zonal Wind JRA25   Run1 185-215   3.209  True   18.774   
                                         310-340   3.218  True   18.783   
                                         80-110    3.340  True   18.905   
  

Well, this is better but also not extremely illustrative / intuitive. It becomes more intuitive if we just look at one parameter that we are interested in (e.g. RMSE). 

#### 5.4 Extracting the Bias of each model run relative to the observations (GET_BIAS)

Retrieving a table that illustrates the Bias of each run for each flagged variable is straight forward. We just extract the `Bias` column from our flagged frame:

In [26]:
bias = selection_unstacked["Bias"]
bias.head()

Run                                    Run1                  Run2   Run3  \
Years                               185-215 310-340 80-110 90-120   1-20   
Variable         Description                                               
CLDTOT_ISCCP     Total cloud cover    1.744   1.435  2.157  3.947  1.786   
FLDS_ISCCP       LW down SRF         10.514  11.499  5.162  4.507 -1.799   
FLNS_ISCCP       LW net SRF           6.824   6.847  7.167  6.925  9.699   
FLUTC_CERES-EBAF LW up Top Clearsky   1.039   1.426 -0.786 -1.182 -0.986   
FLUT_CERES-EBAF  LW up Top            2.398   2.928  0.660 -0.832  1.704   

Run                                   Run4  
Years                               71-100  
Variable         Description                
CLDTOT_ISCCP     Total cloud cover  -3.179  
FLDS_ISCCP       LW down SRF        -5.066  
FLNS_ISCCP       LW net SRF          6.394  
FLUTC_CERES-EBAF LW up Top Clearsky -4.268  
FLUT_CERES-EBAF  LW up Top          -1.426

#### 5.5 Computing RMSE relative error (GET_RMSE_REL)

In the following we extract the subset containing the *RSME* information of the flagged variables for all runs in order to compute the relative error for each run based on the average *RMSE* of all runs:

$$\frac{RMSE_{Run}\,-\,\overline{RMSE_{All\,Runs}}}{\overline{RMSE_{All\,Runs}}}$$


In [27]:
rmse = selection_unstacked["RMSE"]
rmse

Run                                      Run1                    Run2    Run3  \
Years                                 185-215 310-340  80-110  90-120    1-20   
Variable          Description                                                   
CLDTOT_ISCCP      Total cloud cover    12.992  13.078  12.869  12.485  11.881   
FLDS_ISCCP        LW down SRF          16.891  17.664  16.720  15.278  15.351   
FLNS_ISCCP        LW net SRF           14.098  14.174  14.516  13.988  14.953   
FLUTC_CERES-EBAF  LW up Top Clearsky    4.662   4.738   5.670   4.778   4.778   
FLUT_CERES-EBAF   LW up Top             7.188   7.467   7.499   6.598   6.169   
FSDS_ISCCP        SW down SRF          15.915  16.082  16.048  15.421  15.089   
FSNS_ISCCP        SW net SRF           13.587  13.727  13.705  13.068  12.632   
FSNTOAC_CERES     SW net TOA clearsky  15.977  15.710  18.506  17.609  17.432   
FSNTOA_CERES      SW net TOA           12.096  12.125  12.314  12.711  10.795   
LHFLX_JRA25       Lat Heat Flux        14.947  15.116  15.587  15.153  14.578   
LWCF_CERES-EBAF   LW Cloud Forc         5.860   5.980   6.018   5.673   5.683   
PRECT_GPCP        Precipitation         1.043   1.069   1.055   0.975   0.891   
PREH2O_ERA40      Precipitable water    4.153   4.430   3.351   2.373   1.920   
PSL_JRA25         SeaLev pressure       1.825   1.939   1.940   1.695   1.512   
RESSURF           SRF net flux            NaN     NaN     NaN     NaN     NaN   
RESTOA_CERES-EBAF TOA  net flux         9.011   8.937   8.872   9.827   8.358   
RESTOM            TOmodel net flux        NaN     NaN     NaN     NaN     NaN   
SHFLX_JRA25       Sens Heat Flux       11.051  11.138  11.551  11.202  11.926   
SST_HADISST_PI    SST preindustrial     2.458   2.606   2.135   1.732   1.223   
SWCF_CERES-EBAF   SW Cloud Forc        10.447  10.563  10.393   9.973   8.854   
TREFHT_JRA25      2m Temp               2.538   2.634   2.665   2.147   1.955   
TS_LAND_NCEP      2m Temp land          5.068   5.229   4.164   3.944   3.132   
TS_NCEP           2m Temp               3.382   3.529   3.004   2.663   2.320   
U_200_JRA25       Zonal Wind JRA25      4.334   4.348   4.335   3.342   2.852   
Z3_500_JRA25      Geop Height  JRA25    0.391   0.423   0.335   0.199   0.192   

Run                                      Run4  
Years                                  71-100  
Variable          Description                  
CLDTOT_ISCCP      Total cloud cover    11.323  
FLDS_ISCCP        LW down SRF          14.450  
FLNS_ISCCP        LW net SRF           11.967  
FLUTC_CERES-EBAF  LW up Top Clearsky    6.042  
FLUT_CERES-EBAF   LW up Top             6.855  
FSDS_ISCCP        SW down SRF          13.380  
FSNS_ISCCP        SW net SRF           12.711  
FSNTOAC_CERES     SW net TOA clearsky  18.458  
FSNTOA_CERES      SW net TOA           12.307  
LHFLX_JRA25       Lat Heat Flux        17.176  
LWCF_CERES-EBAF   LW Cloud Forc         6.156  
PRECT_GPCP        Precipitation         1.162  
PREH2O_ERA40      Precipitable water    2.147  
PSL_JRA25         SeaLev pressure       1.333  
RESSURF           SRF net flux            NaN  
RESTOA_CERES-EBAF TOA  net flux         8.842  
RESTOM            TOmodel net flux        NaN  
SHFLX_JRA25       Sens Heat Flux       11.761  
SST_HADISST_PI    SST preindustrial     1.059  
SWCF_CERES-EBAF   SW Cloud Forc        10.324  
TREFHT_JRA25      2m Temp               2.316  
TS_LAND_NCEP      2m Temp land          2.767  
TS_NCEP           2m Temp               1.990  
U_200_JRA25       Zonal Wind JRA25      2.444  
Z3_500_JRA25      Geop Height  JRA25    0.280

##### Side comment: Series vs. unstacked Multiindex Dataframes

As you can see in the previous output, we have extracted ***ONE*** variable from the ***UNSTACKED*** dataframe. Now, this is still a pandas ``Dataframe`` since it is *tabular* data. 

In [28]:
print("Extracted table is Dataframe since it is a wide table: {}".format(isinstance(rmse, pd.DataFrame)))

Extracted table is Dataframe since it is a wide table: True


In [29]:
rmse_mean = rmse.mean(axis=1, skipna=True)
#Note that the created object is a Series and not a Dataframe
rmse_mean.head()

Variable          Description       
CLDTOT_ISCCP      Total cloud cover     12.438000
FLDS_ISCCP        LW down SRF           16.059000
FLNS_ISCCP        LW net SRF            13.949333
FLUTC_CERES-EBAF  LW up Top Clearsky     5.111333
FLUT_CERES-EBAF   LW up Top              6.962667
dtype: float64

The next step is (semi) straight forward (we have to use the `div` and `subtract` methods of the Dataframe rather than `/` and `-` operators in order to specify that we want to apply them in the horizontal and not in the vertical direction.

In [30]:
rmse_err_rel = rmse.subtract(rmse_mean, axis=0).div(rmse_mean, axis=0)
rmse_err_rel

Run                                        Run1                          Run2  \
Years                                   185-215   310-340    80-110    90-120   
Variable          Description                                                   
CLDTOT_ISCCP      Total cloud cover    0.044541  0.051455  0.034652  0.003779   
FLDS_ISCCP        LW down SRF          0.051809  0.099944  0.041161 -0.048633   
FLNS_ISCCP        LW net SRF           0.010658  0.016106  0.040623  0.002772   
FLUTC_CERES-EBAF  LW up Top Clearsky  -0.087909 -0.073040  0.109300 -0.065215   
FLUT_CERES-EBAF   LW up Top            0.032363  0.072434  0.077030 -0.052375   
FSDS_ISCCP        SW down SRF          0.038669  0.049568  0.047349  0.006428   
FSNS_ISCCP        SW net SRF           0.026338  0.036913  0.035251 -0.012867   
FSNTOAC_CERES     SW net TOA clearsky -0.075512 -0.090962  0.070825  0.018921   
FSNTOA_CERES      SW net TOA           0.003151  0.005556  0.021231  0.054155   
LHFLX_JRA25       Lat Heat Flux       -0.031062 -0.020107  0.010426 -0.017708   
LWCF_CERES-EBAF   LW Cloud Forc       -0.005937  0.014419  0.020865 -0.037659   
PRECT_GPCP        Precipitation        0.010169  0.035351  0.021792 -0.055690   
PREH2O_ERA40      Precipitable water   0.356155  0.446609  0.094264 -0.225101   
PSL_JRA25         SeaLev pressure      0.068918  0.135689  0.136275 -0.007224   
RESSURF           SRF net flux              NaN       NaN       NaN       NaN   
RESTOA_CERES-EBAF TOA  net flux        0.004067 -0.004179 -0.011421  0.094991   
RESTOM            TOmodel net flux          NaN       NaN       NaN       NaN   
SHFLX_JRA25       Sens Heat Flux      -0.033849 -0.026243  0.009865 -0.020647   
SST_HADISST_PI    SST preindustrial    0.315259  0.394453  0.142424 -0.073219   
SWCF_CERES-EBAF   SW Cloud Forc        0.035142  0.046636  0.029792 -0.011824   
TREFHT_JRA25      2m Temp              0.068257  0.108664  0.121712 -0.096317   
TS_LAND_NCEP      2m Temp land         0.251152  0.290899  0.027979 -0.026333   
TS_NCEP           2m Temp              0.201563  0.253790  0.067267 -0.053884   
U_200_JRA25       Zonal Wind JRA25     0.200831  0.204710  0.201108 -0.074024   
Z3_500_JRA25      Geop Height  JRA25   0.289011  0.394505  0.104396 -0.343956   

Run                                        Run3      Run4  
Years                                      1-20    71-100  
Variable          Description                              
CLDTOT_ISCCP      Total cloud cover   -0.044782 -0.089645  
FLDS_ISCCP        LW down SRF         -0.044087 -0.100193  
FLNS_ISCCP        LW net SRF           0.071951 -0.142110  
FLUTC_CERES-EBAF  LW up Top Clearsky  -0.065215  0.182079  
FLUT_CERES-EBAF   LW up Top           -0.113989 -0.015463  
FSDS_ISCCP        SW down SRF         -0.015239 -0.126774  
FSNS_ISCCP        SW net SRF          -0.045801 -0.039834  
FSNTOAC_CERES     SW net TOA clearsky  0.008680  0.068048  
FSNTOA_CERES      SW net TOA          -0.104744  0.020650  
LHFLX_JRA25       Lat Heat Flux       -0.054982  0.113433  
LWCF_CERES-EBAF   LW Cloud Forc       -0.035963  0.044275  
PRECT_GPCP        Precipitation       -0.137046  0.125424  
PREH2O_ERA40      Precipitable water  -0.373027 -0.298901  
PSL_JRA25         SeaLev pressure     -0.114408 -0.219250  
RESSURF           SRF net flux              NaN       NaN  
RESTOA_CERES-EBAF TOA  net flux       -0.068695 -0.014764  
RESTOM            TOmodel net flux          NaN       NaN  
SHFLX_JRA25       Sens Heat Flux       0.042650  0.028224  
SST_HADISST_PI    SST preindustrial   -0.345581 -0.433336  
SWCF_CERES-EBAF   SW Cloud Forc       -0.122700  0.022955  
TREFHT_JRA25      2m Temp             -0.177131 -0.025184  
TS_LAND_NCEP      2m Temp land        -0.226794 -0.316903  
TS_NCEP           2m Temp             -0.175746 -0.292989  
U_200_JRA25       Zonal Wind JRA25    -0.209790 -0.322835  
Z3_500_JRA25      Geop Height  JRA25  -0.367033 -0.076923

### 5.6 Inserting column of RMSE relative error into original table (INSERT_RMSE_REL_ORIG)

If we want, we can now add the typical RMSE to our original dataframe (containing the only flagged data, since it was computed from this). 

**Note: this is just illustrative and not used in the following section**

First we have to stack it:

In [31]:
stacked = rmse_err_rel.stack(level=(0,1)).reorder_levels(order=(2,3,0,1))
stacked.head()

Run   Years    Variable      Description      
Run1  185-215  CLDTOT_ISCCP  Total cloud cover    0.044541
      310-340  CLDTOT_ISCCP  Total cloud cover    0.051455
      80-110   CLDTOT_ISCCP  Total cloud cover    0.034652
Run2  90-120   CLDTOT_ISCCP  Total cloud cover    0.003779
Run3  1-20     CLDTOT_ISCCP  Total cloud cover   -0.044782
dtype: float64

In [32]:
selection["RMSE_ERR"] = stacked
selection

Flag     Model       Obs  \
Run  Years   Variable          Description                                     
Run1 185-215 CLDTOT_ISCCP      Total cloud cover    True    68.543    66.800   
             FLDS_ISCCP        LW down SRF          True   353.861   343.347   
             FLNS_ISCCP        LW net SRF           True    56.249    49.425   
             FLUTC_CERES-EBAF  LW up Top Clearsky   True   267.090   266.051   
             FLUT_CERES-EBAF   LW up Top            True   241.972   239.574   
             FSDS_ISCCP        SW down SRF          True   190.458   189.390   
             FSNS_ISCCP        SW net SRF           True   166.962   165.893   
             FSNTOAC_CERES     SW net TOA clearsky  True   290.330   294.702   
             FSNTOA_CERES      SW net TOA           True   244.525   244.691   
             LHFLX_JRA25       Lat Heat Flux        True    87.926    87.935   
             LWCF_CERES-EBAF   LW Cloud Forc        True    25.119    26.476   
             PRECT_GPCP        Precipitation        True     3.010     2.674   
             PREH2O_ERA40      Precipitable water   True    28.091    24.948   
             PSL_JRA25         SeaLev pressure      True  1011.771  1011.280   
             RESSURF           SRF net flux         True     0.474     0.000   
             RESTOA_CERES-EBAF TOA  net flux        True     2.553     0.992   
             RESTOM            TOmodel net flux     True     0.468     0.000   
             SHFLX_JRA25       Sens Heat Flux       True    22.314    19.372   
             SST_HADISST_PI    SST preindustrial    True    22.582    20.365   
             SWCF_CERES-EBAF   SW Cloud Forc        True   -45.805   -47.071   
             TREFHT_JRA25      2m Temp              True   289.039   287.605   
             TS_LAND_NCEP      2m Temp land         True   284.920   281.155   
             TS_NCEP           2m Temp              True   290.262   287.683   
             U_200_JRA25       Zonal Wind JRA25     True    18.774    15.565   
             Z3_500_JRA25      Geop Height  JRA25   True    56.879    56.601   
     310-340 CLDTOT_ISCCP      Total cloud cover    True    68.234    66.800   
             FLDS_ISCCP        LW down SRF          True   354.846   343.347   
             FLNS_ISCCP        LW net SRF           True    56.272    49.425   
             FLUTC_CERES-EBAF  LW up Top Clearsky   True   267.477   266.051   
             FLUT_CERES-EBAF   LW up Top            True   242.502   239.574   
...                                                  ...       ...       ...   
Run3 1-20    TREFHT_JRA25      2m Temp              True   287.342   287.605   
             TS_LAND_NCEP      2m Temp land         True   282.392   281.155   
             TS_NCEP           2m Temp              True   288.559   287.683   
             U_200_JRA25       Zonal Wind JRA25     True    17.390    15.565   
             Z3_500_JRA25      Geop Height  JRA25   True    56.511    56.601   
Run4 71-100  CLDTOT_ISCCP      Total cloud cover    True    63.621    66.800   
             FLDS_ISCCP        LW down SRF          True   338.280   343.347   
             FLNS_ISCCP        LW net SRF           True    55.819    49.425   
             FLUTC_CERES-EBAF  LW up Top Clearsky   True   261.783   266.051   
             FLUT_CERES-EBAF   LW up Top            True   238.148   239.574   
             FSDS_ISCCP        SW down SRF          True   187.801   189.390   
             FSNS_ISCCP        SW net SRF           True   163.679   165.893   
             FSNTOAC_CERES     SW net TOA clearsky  True   287.999   294.702   
             FSNTOA_CERES      SW net TOA           True   239.677   244.691   
             LHFLX_JRA25       Lat Heat Flux        True    87.904    87.935   
             LWCF_CERES-EBAF   LW Cloud Forc        True    23.635    26.476   
             PRECT_GPCP        Precipitation        True     3.003     2.674   
             PREH2O_ERA40      Precipitable water   

### 6. Conditional formatting of tables (Dataframes) (VISUALISE)

This section illustrates, how we can perform conditional formatting of the color tables. As discussed above, we can apply background colour gradients to the data. In the example above we had a multiindex data type specifying model run, year-range and variable in stacked format (long table) and the four data columns specifying results from model and observation as well as bias and RMSE. 

Now, in the following we illustrate how we can apply this colour highlighting for the two unstacked tables that we just created and that contain Bias and relative error. 

Starting with the Bias data, we show an example that does not work for our purposes (since it only allows for conditional formatting of either rows or columns.

#### 6.1 NOT how we want it (using the style method `background_gradient`) (VIS_WRONG)

The most straight forward example for conditional formatting of a Dataframe is shown in the following. In the example we use the `Bias` table and, similar to the example above, apply a value based colormap. Here, we use a *diverging colormap (bwr)* which has white as center color. Like in the example above, we use the style method `background_gradient` which can perform the formatting either in a **rowwise** or **columnwise** manner (using input argument `axis=1` or `axis=0`, respectively). 

Note, however, that this is not what we are aiming for in this example, rather, we want the colour formatting to be applied based on the values available the **whole table** and not individually for **columns** or **rows** (which is done in the next section). Nonetheless, in the cell below we show what we get if we use the method `backgroun_gradient`.  

Again, we use the `low` and `high` parameters to specify the colorrange that we use to map the values (see above).

In [41]:
bias.style.background_gradient(cmap="bwr", low=0.5, high=0.5, axis=1).highlight_null("white")

In [34]:
bias

Run                                      Run1                  Run2   Run3  \
Years                                 185-215 310-340 80-110 90-120   1-20   
Variable          Description                                                
CLDTOT_ISCCP      Total cloud cover     1.744   1.435  2.157  3.947  1.786   
FLDS_ISCCP        LW down SRF          10.514  11.499  5.162  4.507 -1.799   
FLNS_ISCCP        LW net SRF            6.824   6.847  7.167  6.925  9.699   
FLUTC_CERES-EBAF  LW up Top Clearsky    1.039   1.426 -0.786 -1.182 -0.986   
FLUT_CERES-EBAF   LW up Top             2.398   2.928  0.660 -0.832  1.704   
FSDS_ISCCP        SW down SRF           1.068   1.228  1.216 -1.721  3.200   
FSNS_ISCCP        SW net SRF            1.070   1.365 -0.017 -2.145  2.630   
FSNTOAC_CERES     SW net TOA clearsky  -4.373  -4.183 -5.747 -4.988 -5.051   
FSNTOA_CERES      SW net TOA           -0.167   0.223 -1.724 -3.937 -0.338   
LHFLX_JRA25       Lat Heat Flux        -0.009   0.434 -1.695 -2.692 -2.503   
LWCF_CERES-EBAF   LW Cloud Forc        -1.357  -1.501 -1.446 -0.349 -2.689   
PRECT_GPCP        Precipitation         0.336   0.352  0.276  0.244  0.246   
PREH2O_ERA40      Precipitable water    3.143   3.437  1.922  0.785 -0.511   
PSL_JRA25         SeaLev pressure       0.491   0.513  0.448  0.335  0.004   
RESSURF           SRF net flux          0.474   0.337  0.643 -0.075  0.974   
RESTOA_CERES-EBAF TOA  net flux         1.561   1.420  1.741  1.021  2.083   
RESTOM            TOmodel net flux      0.468   0.327  0.647 -0.075  0.987   
SHFLX_JRA25       Sens Heat Flux        2.942   2.907  3.028  2.857  3.621   
SST_HADISST_PI    SST preindustrial     2.217   2.385  1.756  1.358  0.782   
SWCF_CERES-EBAF   SW Cloud Forc         1.266   1.466  1.083 -1.888  1.774   
TREFHT_JRA25      2m Temp               1.434   1.615  0.431  0.344 -0.263   
TS_LAND_NCEP      2m Temp land          3.765   3.945  2.181  2.153  1.237   
TS_NCEP           2m Temp               2.579   2.767  1.564  1.518  0.876   
U_200_JRA25       Zonal Wind JRA25      3.209   3.218  3.340  2.304  1.825   
Z3_500_JRA25      Geop Height  JRA25    0.278   0.317  0.113 -0.014 -0.090   

Run                                     Run4  
Years                                 71-100  
Variable          Description                 
CLDTOT_ISCCP      Total cloud cover   -3.179  
FLDS_ISCCP        LW down SRF         -5.066  
FLNS_ISCCP        LW net SRF           6.394  
FLUTC_CERES-EBAF  LW up Top Clearsky  -4.268  
FLUT_CERES-EBAF   LW up Top           -1.426  
FSDS_ISCCP        SW down SRF         -1.589  
FSNS_ISCCP        SW net SRF          -2.214  
FSNTOAC_CERES     SW net TOA clearsky -6.703  
FSNTOA_CERES      SW net TOA          -5.015  
LHFLX_JRA25       Lat Heat Flux       -0.031  
LWCF_CERES-EBAF   LW Cloud Forc       -2.841  
PRECT_GPCP        Precipitation        0.329  
PREH2O_ERA40      Precipitable water  -0.975  
PSL_JRA25         SeaLev pressure      0.002  
RESSURF           SRF net flux        -0.489  
RESTOA_CERES-EBAF TOA  net flux        0.537  
RESTOM            TOmodel net flux    -0.489  
SHFLX_JRA25       Sens Heat Flux       1.074  
SST_HADISST_PI    SST preindustrial   -0.108  
SWCF_CERES-EBAF   SW Cloud Forc       -1.252  
TREFHT_JRA25      2m Temp             -1.098  
TS_LAND_NCEP      2m Temp land        -0.524  
TS_NCEP           2m Temp             -0.346  
U_200_JRA25       Zonal Wind JRA25     1.006  
Z3_500_JRA25      Geop Height  JRA25  -0.254

Now, this worked nicely but there are mainly two problems with this representation:

1. As mentioned above, one problem here is that the colour coding can only be performed row or column wise using the input parameter `axis` (and not based on the values of the whole table, see [here](https://pandas.pydata.org/pandas-docs/stable/style.html#Building-Styles-Summary) for details)
2. If we use the symmetric colormap as is (i.e. center colour is white), then, the color white will be mapped to the midpoint value of the considered value range (e.g. min=-2, max=4 => (4 - -2)/2 = 3 => 1 == white). However, what we want is a *shifter diverging colormap* that ensures that the value 0 is mapped white, even if min != -max.
3. Further, we might wish to have control over the number of significant digits that are displayed in the table

All these problems will be solved in the following.

#### 6.2 How we want it (VIS_RIGHT)

In the following, we use a custom display method `my_table_display` (that is defined in [helper_funcs.py](https://github.com/jgliss/my_py3_scripts/blob/master/notebooks/helper_funcs.py)) in order to perform colour formatting considering all rows and columns at the same time and furthermore, using a diverging colour map that is dynamically shifted such that value 0 corresponds to the colour white (method `shifted_color_map`) also if `-vmin != vmax` (like usually).

In [35]:
from helper_funcs import my_table_display
my_table_display(bias)

Now for the typical RMSE error

In [36]:
my_table_display(rmse_err_rel)

/home/jonasg/anaconda3/lib/python3.6/site-packages/matplotlib/colors.py:489: RuntimeWarning: invalid value encountered in less
  np.copyto(xa, -1, where=xa < 0.0)


### 7. Concatenate and save results (Bias and typical RMSE) as table (EXPORT)

In the following, the two result tables ``bias_table`` and ``typical_rmse`` are merged into one result table and then saved both as excel table and as csv file.

In [60]:
result = pd.concat([bias, rmse_err_rel],axis=1, keys=["Bias", "RMSE relative Error"])
result

Bias                               \
Run                                      Run1                  Run2   Run3   
Years                                 185-215 310-340 80-110 90-120   1-20   
Variable          Description                                                
CLDTOT_ISCCP      Total cloud cover     1.744   1.435  2.157  3.947  1.786   
FLDS_ISCCP        LW down SRF          10.514  11.499  5.162  4.507 -1.799   
FLNS_ISCCP        LW net SRF            6.824   6.847  7.167  6.925  9.699   
FLUTC_CERES-EBAF  LW up Top Clearsky    1.039   1.426 -0.786 -1.182 -0.986   
FLUT_CERES-EBAF   LW up Top             2.398   2.928  0.660 -0.832  1.704   
FSDS_ISCCP        SW down SRF           1.068   1.228  1.216 -1.721  3.200   
FSNS_ISCCP        SW net SRF            1.070   1.365 -0.017 -2.145  2.630   
FSNTOAC_CERES     SW net TOA clearsky  -4.373  -4.183 -5.747 -4.988 -5.051   
FSNTOA_CERES      SW net TOA           -0.167   0.223 -1.724 -3.937 -0.338   
LHFLX_JRA25       Lat Heat Flux        -0.009   0.434 -1.695 -2.692 -2.503   
LWCF_CERES-EBAF   LW Cloud Forc        -1.357  -1.501 -1.446 -0.349 -2.689   
PRECT_GPCP        Precipitation         0.336   0.352  0.276  0.244  0.246   
PREH2O_ERA40      Precipitable water    3.143   3.437  1.922  0.785 -0.511   
PSL_JRA25         SeaLev pressure       0.491   0.513  0.448  0.335  0.004   
RESSURF           SRF net flux          0.474   0.337  0.643 -0.075  0.974   
RESTOA_CERES-EBAF TOA  net flux         1.561   1.420  1.741  1.021  2.083   
RESTOM            TOmodel net flux      0.468   0.327  0.647 -0.075  0.987   
SHFLX_JRA25       Sens Heat Flux        2.942   2.907  3.028  2.857  3.621   
SST_HADISST_PI    SST preindustrial     2.217   2.385  1.756  1.358  0.782   
SWCF_CERES-EBAF   SW Cloud Forc         1.266   1.466  1.083 -1.888  1.774   
TREFHT_JRA25      2m Temp               1.434   1.615  0.431  0.344 -0.263   
TS_LAND_NCEP      2m Temp land          3.765   3.945  2.181  2.153  1.237   
TS_NCEP           2m Temp               2.579   2.767  1.564  1.518  0.876   
U_200_JRA25       Zonal Wind JRA25      3.209   3.218  3.340  2.304  1.825   
Z3_500_JRA25      Geop Height  JRA25    0.278   0.317  0.113 -0.014 -0.090   

                                             RMSE relative Error            \
Run                                     Run4                Run1             
Years                                 71-100             185-215   310-340   
Variable          Description                                                
CLDTOT_ISCCP      Total cloud cover   -3.179            0.044541  0.051455   
FLDS_ISCCP        LW down SRF         -5.066            0.051809  0.099944   
FLNS_ISCCP        LW net SRF           6.394            0.010658  0.016106   
FLUTC_CERES-EBAF  LW up Top Clearsky  -4.268           -0.087909 -0.073040   
FLUT_CERES-EBAF   LW up Top           -1.426            0.032363  0.072434   
FSDS_ISCCP        SW down SRF         -1.589            0.038669  0.049568   
FSNS_ISCCP        SW net SRF          -2.214            0.026338  0.036913   
FSNTOAC_CERES     SW net TOA clearsky -6.703           -0.075512 -0.090962   
FSNTOA_CERES      SW net TOA          -5.015            0.003151  0.005556   
LHFLX_JRA25       Lat Heat Flux       -0.031           -0.031062 -0.020107   
LWCF_CERES-EBAF   LW Cloud Forc       -2.841           -0.005937  0.014419   
PRECT_GPCP        Precipitation        0.329            0.010169  0.035351   
PREH2O_ERA40      Precipitable water  -0.975            0.356155  0.446609   
PSL_JRA25         SeaLev pressure      0.002            0.068918  0.135689   
RESSURF           SRF net flux        -0.489                 NaN       NaN   
RESTOA_CERES-EBAF TOA  net flux        0.537            0.004067 -0.004179   
RESTOM            TOmodel net flux    -0.489                 NaN       NaN   
SHFLX_JRA25       Sens Heat Flux       1.074           -0.033849 -0.026243   
SST_HADISST_PI    SST preindustrial   -0.108            0.3152

Now save both tables as excel file.

In [38]:
writer = pd.ExcelWriter('{}/result.xlsx'.format(output_dir))
df.to_excel(writer)
writer.save()